# Imports

In [10]:
import os
import sys
import time
import logging
import importlib
import datetime
import re
import json
import glob
import gzip
import sqlite3
from pathlib import Path
from IPython.display import display
import numpy as np
from numpy.lib.recfunctions import join_by
from numpy.core.records import fromarrays
import matplotlib.pyplot as plt
from matplotlib.pyplot import pcolormesh
import pandas as pd
import aiohttp
import requests
import math
import itertools
from functools import lru_cache
#@lru_cache(maxsize=None)

from edcompanion import eddbreader, edsm_api
from edcompanion.events import edc_read_journal, edc_track_journal
from edcompanion.timetools import make_datetime, make_naive_utc
from edcompanion.edsm_api import get_edsm_info, distance_between_systems

def prettyprint(item):
    print(json.dumps(item, indent=4, sort_keys=False))
    

plt.rcParams["figure.figsize"] = (16,12)
pd.options.display.max_colwidth = 150


# Some vars 
logpath = "/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous"
system_name = None
connection = sqlite3.connect('./data/ed.db')
connection.close()
result = None


In [25]:
with sqlite3.connect('./data/ed.db') as connection:
    result = connection.execute(f"SELECT * FROM systems WHERE name like 'Ix%'")

In [26]:
result.fetchone()

('Ixilalana', 57.25, 155.09375, 37.15625)

In [27]:
result.fetchall()

[('Ix Ching', 33.34375, -78.15625, 171.8125),
 ('IX Orionis', 606.5625, -444.34375, -1097.5),
 ('Ixbalanque', 77.5, -55.40625, -96.8125),
 ('Ixchertha', 3.21875, -97.375, 88.3125),
 ('Ixilalonta', 131.71875, -9.5, -1.46875),
 ('Ix Chup', 96.71875, -180.625, 93.4375),
 ('Ix Tun', -32.65625, 83.84375, 61.15625),
 ('Ix Tub Tun', 10.9375, -178.15625, -13.46875),
 ('Ix Chowagit', 122.625, -160.5, 47.40625),
 ('Ix Chau', -84.9375, -82.0625, -115.125),
 ('Ix Chita', -14.21875, -55.3125, 141.25),
 ('Ix Chons', 94.46875, -173.53125, 1.84375),
 ('Ixtabru', 124.28125, -239.625, -21.0),
 ('Ixbakah', 3.75, -144.53125, -67.0),
 ('Ixbaksha', -146.03125, -39.65625, -20.53125),
 ('Ixcateshai', 96.34375, -120.25, -62.40625),
 ('Ix Chuntin', 117.25, -79.4375, 122.84375),
 ('Ixbakald', 81.25, -93.125, -39.28125),
 ('Ix Chuacae', -14.90625, -51.96875, 140.65625),
 ('Ixilak', 71.53125, -173.5, 154.53125),
 ('Ixbalan', 87.8125, -81.34375, -136.25),
 ('Ixil', -22.03125, -183.0, 68.0),
 ('Ix Chim', -78.71875, 

In [51]:
importlib.reload(eddbreader)

def structure(thing, prefix):
    if isinstance(thing, dict):
        return '\n'.join([f"{prefix}-{str(k):20}{str(type(v)):24}\n{structure(v, prefix+'   ') if v else ''}" for k, v in thing.items()])


    elif isinstance(thing, list):
        return '\n'.join([f"{prefix}-{str(type(v)):24}\n{structure(v, prefix+'   ') if v else ''}" for v in thing])
    else:
        return ''
        return f"{prefix}{str(thing):42}\n"


eddbfile = './data/galaxy_1day.json.gz'
keys = {}
count = 1
item = None

for item in eddbreader.edc_dbfilereader(eddbfile, True):
    if not [k for k in item.get('bodies',[]) if 'ary' in k.get('type') ]:
        continue

    print(structure(item,'   '))

    count -= 1
    if not count > 0:
        break

<module 'edcompanion.eddbreader' from '/home/fenke/repos/EDCompanion/edcompanion/eddbreader.py'>

In [52]:

systems = [(
        item.get('id64'),
        *item.get('coord', [None,None,None]),
        item.get('name'),
        item.get('bodyCount'),
    )
    for item in eddbreader.edc_dbfilereader(eddbfile, True)
]


Reading ./data/galaxy_1day.json.gz, 489.8 Mb in approx 62 chunks
48/62	77.42%,    721 /s,     83925 systems,  33.9 seconds remaining
Empty chunk -> Done! Imported 83925 systems in 116.3 seconds

116.25737712499995 seconds 83925 systems, per system 1385.25 us

In [ ]:
import sys
import time
from pathlib import Path
import gzip
import json


In [68]:
#filename = './data/galaxy_7days.json.gz'
#filename = './data/systemsWithCoordinates7days.json.gz'
filename = './data/systemsWithCoordinates.json.gz'
assert '.gz' in filename
filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(8*filesize/chunksize) + 1
print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

count = 0
system_count = 0
item = None
line = ''
start = time.process_time()
#try:
with gzip.open(filename.replace('.json.gz','_systems.jsonl.gz' if 'systems' not in filename else '.jsonl.gz'), 'wt') as jsonl_file:

    with gzip.open(filename, 'rt') as jsonfile:

        while True:
            count += 1
            data = [
                json.dumps({
                    k:v
                    for k, v in json.loads(line.rstrip(',\r\n')).items()
                    if k in ['id64', 'name', 'coords']
                })  + '\n'
                for line in jsonfile.readlines(chunksize)
                if len(line) > 4
                
            ]
            system_count += len(data)
            if data:
                jsonl_file.writelines(data)
            else:
                break

            sys.stdout.write(f"\r{count}/{est_count}\t{100*count/est_count:3.2f}%, {int(system_count / (time.process_time() - start)):6} /s, {system_count:9} systems, {((est_count - count) * (time.process_time() - start)/count):5.1f} seconds remaining")

tpl = (time.process_time() - start)/system_count
sys.stdout.write(f"\n{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us")



Reading ./data/systemsWithCoordinates.json.gz, 2631.1 Mb in approx 329 chunks
172/329	52.28%,  14792 /s,  74498982 systems, 4597.0 seconds remaining
5036.286807084 seconds 74498982 systems, per system 67.6 us

60

## Import data into a local database
### Drop & Create systems table

In [3]:
# deleting the file is a lot faster ...
with sqlite3.connect('./data/ed.db') as connection:

    connection.execute(f"""
    DROP TABLE IF EXISTS systems;
""")


### Create systems table

In [4]:

with sqlite3.connect('./data/ed.db') as connection:

    connection.execute(f"""
    CREATE TABLE IF NOT EXISTS systems (
        name text NOT NULL,
        x DOUBLE PRECISION,
        y DOUBLE PRECISION,
        z DOUBLE PRECISION
    );
""")


### Import main systems data file

In [5]:
#filename = './data/galaxy_7days.json.gz'
#filename = './data/systemsWithCoordinates7days.json.gz'
est_system_count, filename = 7.8e7, './data/systemsWithCoordinates.json.gz'
assert '.gz' in filename
from functools import reduce
filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(8*filesize/chunksize) + 1

print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")
data=[]
count = 0
system_count = 0
item = None
line = ''
start = time.process_time()

def extract_system(line):
    d = json.loads(line.rstrip(',\r\n'))
    return d['name'], *d['coords'].values()

#try:
with sqlite3.connect('./data/ed.db') as connection:

    connection.execute(f"""
        DROP INDEX IF EXISTS systems_x_idx;
    """)
    connection.execute(f"""
        DROP INDEX IF EXISTS systems_y_idx;
    """)    
    connection.execute(f"""
        DROP INDEX IF EXISTS systems_z_idx; 
    """)

    connection.execute(f"""
        DROP INDEX IF EXISTS systems_name_idx; 
    """)
    with gzip.open(filename, 'rt') as jsonfile:

        while True:
            count += 1
            data = [
                extract_system(line)
                for line in jsonfile.readlines(chunksize)
                if len(line) > 4
                
            ]
            system_count += len(data)
            est_count = round(est_system_count / (system_count/count))
            sys.stdout.write(f"\r{count}/{est_count}\t{100*count/est_count:3.2f}%, {int(system_count / (time.process_time() - start)):6} /s, {system_count:9} systems, {((est_count - count) * (time.process_time() - start)/count):5.1f} seconds remaining")

            
            if data:
                connection.executemany(f"""
                    INSERT INTO systems
                    VALUES (?,?,?,?)
                """, data)
                continue

            break

    connection.execute(f"""
        CREATE INDEX IF NOT EXISTS systems_x_idx ON systems (x);
    """)
    connection.execute(f"""
        CREATE INDEX IF NOT EXISTS systems_y_idx ON systems (y);
    """)    
    connection.execute(f"""
        CREATE INDEX IF NOT EXISTS systems_z_idx ON systems (z); 
    """)

tpl = (time.process_time() - start)/system_count
sys.stdout.write(f"\n{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us")



Reading ./data/systemsWithCoordinates.json.gz, 2631.1 Mb in approx 329 chunks
173/181	95.58%,  81896 /s,  74498982 systems,  42.1 seconds remaining
1376.13787072 seconds 74498982 systems, per system 18.47 us

60

#### Setup indexes
##### index name

In [6]:
with sqlite3.connect('./data/ed.db') as connection:
    connection.execute(f"""
        CREATE INDEX IF NOT EXISTS systems_name_idx ON systems (name);
    """)


In [7]:

with sqlite3.connect('./data/ed.db') as connection:
 
    connection.execute("""
    DELETE FROM systems as a
    WHERE   a.rowid <> (SELECT min(b.rowid)
                     FROM   systems as b
                     WHERE  a.name = b.name );"""
    )
    


In [8]:
with sqlite3.connect('./data/ed.db') as connection:

    connection.execute(f"""
        DROP INDEX IF EXISTS systems_name_idx; 
    """)

    connection.execute(f"""
        CREATE UNIQUE INDEX systems_name_idx ON systems (name); 
    """)


##### index coordinates

In [9]:

with sqlite3.connect('./data/ed.db') as connection:

    connection.execute(f"""
        CREATE INDEX IF NOT EXISTS systems_x_idx ON systems (x);
    """)
    connection.execute(f"""
        CREATE INDEX IF NOT EXISTS systems_y_idx ON systems (y);
    """)    
    connection.execute(f"""
        CREATE INDEX IF NOT EXISTS systems_z_idx ON systems (z); 
    """)

In [74]:


with sqlite3.connect('./data/ed.db') as connection:

    connection.execute(f"""
        DROP INDEX IF EXISTS systems_x_idx;
    """)
    connection.execute(f"""
        DROP INDEX IF EXISTS systems_y_idx;
    """)    
    connection.execute(f"""
        DROP INDEX IF EXISTS systems_z_idx; 
    """)

In [ ]:
result = None
with sqlite3.connect('./data/ed.db') as connection:


# Routing

In [ ]:
importlib.reload(edsm_api)
from edcompanion.edsm_api import get_edsm_info

async with aiohttp.ClientSession() as session:
    async with session.get('https://edastro.com/gec/json/all') as req:
        _edastro_poi = await req.json()
        edastro_poi = {
            item.get('galMapSearch'):{
                c:item.get(k)
                for c,k in zip(['coordinates','name','region','type','summary'], ['coordinates','name','region','type','summary'])
            }
            for item in _edastro_poi
        }

In [38]:
sqlite3.paramstyle

'qmark'

In [57]:

def find_system(system, distance=40):

    if isinstance(system, str):
        return connection.execute(
            """
                SELECT s.*, 0 as distance
                FROM systems s
                where s.name = ?
            """, (system,)).fetchone()

    assert len(system) == 3
    coordinates = system
    c20_location = [int(20*math.floor(v/20)) for v in coordinates]
    side = int(20*math.floor(distance/20))
    q1 = connection.execute(
        "SELECT systems.*, sqrt((x-?)*(x-?) + (y-?)*(y-?) + (z-?)*(z-?)) as distance "+
        "FROM systems "+
        "WHERE x>=? AND x<? AND  y>=? AND y<?  AND  z>=? AND z<?  AND sqrt((x-?)*(x-?) + (y-?)*(y-?) + (z-?)*(z-?)) < ?"+
        "ORDER BY distance", (
            *[c for p in zip(coordinates, coordinates) for c in p], # for calculated distance in select
            *[d for c in coordinates for d in [c-40, c+40]], 
            *[c for p in zip(coordinates, coordinates) for c in p], # cube and coordinates
            distance
        ))
    if not q1.rowcount > 0:
        return q1.fetchone()
    return find_system(q1[0].get("name"))


In [58]:
find_system('Ix')

('Ix', -65.21875, 7.75, -111.03125, 0)

In [59]:
find_system([-65.21875, 7.75, -111.03125])

('Ix', -65.21875, 7.75, -111.03125, 0.0)

In [ ]:
targetsystems = {L.split('\t')[0].replace('#','DRMG'):L.split('\t')[1].split('(')[0].strip() for L in '''
#1	HIP 117029		
0.00%
0.00%
#2	Drojia YW-B d13-4	4,377.94 ly	
0.00%
0.00%
#3	Lysooh WT-R b7-0 (Halley's World)	7,658.41 ly	
0.00%
0.00%
#4	Plaa Ain FF-Z d76 (Rekohu Project)	10,162.39 ly	
0.00%
0.00%
#5	Blaa Hypai LA-J c11-3	13,299.43 ly	
0.00%
0.00%
#6	Floawns XE-R d4-45 (The Three Kings)	15,978.96 ly	
0.00%
0.00%
#7	Mynoaw LC-L d8-1429	21,214.37 ly	
0.00%
0.00%
#8	Egnairs AA-A h72 (Mairon)	26,508.32 ly	
0.00%
0.00%
#9	Hypiae Aurb AA-A g588 (Planet Pancake)	29,889.18 ly	
0.00%
0.00%
#10	Juenae XZ-G d10-651 (Red River Run)	34,771.56 ly	
0.00%
0.00%
#11	Sagittarius A*	36,288.63 ly	
0.00%
0.00%
#12	Hypio Proo VE-Q e5-1485	38,824.33 ly	
0.00%
0.00%
#13	Pho Aoscs OS-U f2-26 (Black Fields)	43,891.03 ly	
0.00%
0.00%
#14	Athaip WR-H d11-7577	48,599.27 ly	
0.00%
0.00%
#15	Dryau Scraa AA-A h747 (The Shiner)	54,319.41 ly	
0.00%
0.00%
#16	Chroabs TI-S d4-58 (Goliath)	58,392.41 ly	
0.00%
0.00%
#17	Ellairb SJ-B b42-10 (Lair of Unicorns)	63,416.44 ly	
0.00%
0.00%
#18	Plaa Aescs BK-Q d5-60	66,359.12 ly	
0.00%
0.00%
#19	Bleia Dryiae XJ-R e4-1	69,286.00 ly	
0.00%
0.00%
#20	Kaititja	74,359.19 ly	
0.00%
0.00%
'''.splitlines() if '#' in L}

In [ ]:
## interesting systems
targetsystems.update(dict(
    sol='Sol',
    ix='Ix',
    proto_1='Dehoae HH-U e3-14',
    guardian_fsd1='HD 63154',
    guardian_fsd2='Synuefe PX-J c25-8',
    mel_brandon='Luchtaine',
    marsha_hicks='Tir',
    # Verruckte POI
    three_dwarfs='Drojia YW-B d13-4',
    eye_of_fatima='Drojaea DG-E d12-4',
    gravi_nightmare='Drojia XK-D c26-0',
    # Events
    hip22460='HIP 22460',
    running_man='HIP 23759',
    fc_hip22460='Pleiades Sector CW-U b3-2',
    uia_3='Synuefai XI-F c2', #[-243,-170,-1033],
    # Asteroid & deep space bases
    witch_science='HIP 23759',
    orion_tourist='PMD2009 48',
    medusa='Crescent Sector GW-W c1-8',
    anchorage='Rohini',
    new_growth='Pencil Sector EL-Y d5',
    # Nebula / POI
    scorch_red_moon='Blue Hypooe VV-A c2-12',
    cygnus_x1='V1357 Cygni',
    elephant='IC 1396 Sector QI-S d4-9',
    ngc7026='Csi+47-21046',
    ngc7354='Csi+61-22385',
    heartsoul='Hypoae Ain MO-I d9-37',
    siteseeing_01='Phua Bre FB-O e6-257',
    thors_eye="Thor's Eye",
    sagitarius_a='Sagittarius A*',
    annihilator='Great Annihilator',
    four_of_a_kind='Dryao Phylio AA-A h410',
    witch_head='Witch Head Sector BQ-Y d14',
    amundsen='Lyed YJ-I d9-0',
    five_eyes='Phrio Phoea AA-A h12',
    statue_liberty='Statue of Liberty Sector DL-Y d27',
    # Carriers
    artemis='Synuefuae CM-J d10-42',
    gorgon='NGC 7822 Sector BQ-Y d12',
    rocksteady='Prooe Hypue FH-U e3-2',
    paradox='Prai Hypoo TX-B d4', # carrier
    inverness='Thraikoo PS-U e2-4',       # carrier
    maerzenbecher='Hedgo GL-P c5-4',      # carrier
    fuelmoon='Hypo Aeb WK-H b51-0',
    brazillian='Graesms CW-E d11-2',
    sanctuary='Syreadiae JX-F c0',        # carrier
    ngc281='BD+55 191',
    farsight='Heart Sector IR-V b2-0',
    minbari="Eor Aescs UP-N c20-0",
    traikeou='Traikeou WL-I c11-1',
    nearby_station='Prielo TZ-G d10-4',
    # Voyager journey
    caretaker='Byoo Briae TJ-G c24-0',
))

### Verruckte Reise

In [12]:
vri = 0

In [13]:
vri+=1
system_name = targetsystems.get(f'DRMG{vri}')
start_system = await find_system(targetsystems.get(f'DRMG{vri}'))
target_system = await find_system(targetsystems.get(f'DRMG{vri+1}'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))
print(start_system)
print(target_system)


NameError: name 'targetsystems' is not defined

#### Startpoints

In [11]:
system_name = None
for event in edc_track_journal(logpath):
    eventname = event.pop('name')
    if eventname == 'Location' or eventname == 'FSDJump':
        system_name = event.get('Starsystem')
        
# Set starting point vars and initialize route & path vars
start_system = await find_system(system_name)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous'